In [1]:
#include <iostream>
#include <string>
#include <algoviz/SVG.hpp>

// Unsere eigenen Kreationen a lá Mary Shelley
#include "Vogel.hpp"
#include "Layout.hpp"

using namespace std;

AlgoViz::clear();

In file included from input_line_9:1:
./Vogel.hpp:38:124: error: use of undeclared identifier 'spielfeld'
  ...posX, posY, 40, 40, &spielfeld);
                          ^
In file included from input_line_9:2:
./Layout.hpp:20:24: error: no matching constructor for initialization of 'Image'
  ...Image("https://ak.picdn.net/shutterstock/videos/26123588/thumb/9.jpg", 400, 200, 1200, 700, &spielfeld);
     ^     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/sandy/miniconda3/envs/aud/include/algoviz/SVG.hpp:2174:5: note: candidate constructor not viable: no known conversion from 'SVG **' to 'SVG *'
      for 6th argument; remove &
    Image(const std::string path, int x, int y, int w, int h, SVG *view...
    ^
/home/sandy/miniconda3/envs/aud/include/algoviz/SVG.hpp:2208:5: note: candidate constructor not viable: requires single argument 'original', but 6
      arguments were provided
    Image(const Image &original)
    ^
/home/sandy/

In [ ]:
// Spielfeld initialisieren
SVG spielfeld = SVG(800,400);

Layout layout = Layout(spielfeld);
Vogel vogel = Vogel(spielfeld);
Text screenScore;

vogel.hide();

In [ ]:
// Erstellt die nötigen Arrays

Rect barriere1[9];
Rect barriere2[9];

In [ ]:
// Benötigte globale Variablen:

string key = "";

int y;

int rndm;
int rndm1;
int rndm2;
int rndm3;
int tmpRunden;
int geschwindigkeit = 0;
int score;
int counter;

bool reachMid = false;
bool alreadyCrashed = false;
bool ascend = false;

// Eine Variable, um die vertikale Geschwindigkeit zu speichern:
int ascension = 5;

// Startpositionierung
int start = 50;

// Die Iterationen der Hauptschleife unten zählen:
int iteration = 0;



In [ ]:
void reseteBarriere(Rect array[], int rndm, int whichbarrier) {
    
    // Vorher unsichtbare Blöcke werden auf schwarz gesetzt
    
    array[rndm].setFill(0, 0, 0, 1);
    array[rndm].setColor(0, 0, 0, 1);
    array[rndm+1].setColor(0, 0, 0, 1);
    array[rndm+1].setFill(0, 0, 0, 1);
    array[rndm+2].setColor(0, 0, 0, 1);
    array[rndm+2].setFill(0, 0, 0, 1);
    
    // Ein neuer Random-Wert wird erstellt
    
    rndm = (rand() % 4) + 1;
    
    // Welche Barrier soll resettet werden
    
    if (whichbarrier == 1) {
        
        rndm1 = rndm;
        
    } else if(whichbarrier == 2) {
        
        rndm2 = rndm;
    }
    
    // Es werden drei Blöcke unsichtbar gemacht
    
    array[rndm].setFill(0, 0, 0, 0);
    array[rndm].setColor(0, 0, 0, 0);
    array[rndm+1].setFill(0, 0, 0, 0);
    array[rndm+1].setColor(0, 0, 0, 0);
    array[rndm+2].setFill(0, 0, 0, 0);
    array[rndm+2].setColor(0, 0, 0, 0);     
}

In [ ]:
void erstelleBarriere(Rect array[]) {  
     
    for (int block = 0; block < 9; block++ ) {
        
            // Die Blöcke werden sichtbar gemacht
        
            array[block] = Rect(800, 420-start, 50, 50, &spielfeld);
            array[block].setFill(0, 0, 0, 1);
            array[block].setColor(0, 0, 0, 1);
        
             // Position auf der X-Achse wird verschoben
        
            start += 50;
    }

    // Es wird eine zufällige Zahl zwischen 1 und 4 gesucht und dann werden n+0...2 zufällige Blöcke transparent gemacht.
    
    rndm = (rand() % 4) + 1;
     
    array[rndm].setFill(0, 0, 0, 0);
    array[rndm].setColor(0, 0, 0, 0);
    array[rndm+1].setFill(0, 0, 0, 0);
    array[rndm+1].setColor(0, 0, 0, 0);
    array[rndm+2].setFill(0, 0, 0, 0);
    array[rndm+2].setColor(0, 0, 0, 0);

    start = 50;
}

In [ ]:
void vogelAbsturz() {
    
    alreadyCrashed = true;
    key = "x";
    
    for (int block = 0; block < 9; block++ ) {
        
        // zwei Steine transparent stellen
        
        barriere1[block].setFill(0, 0, 0, 0);
        barriere2[block].setFill(0, 0, 0, 0);
        barriere1[block].setColor(0, 0, 0, 0);
        barriere2[block].setColor(0, 0, 0, 0);
    }
            
    layout.flackern();
    layout.gameOver();
    vogel.death();
}

In [ ]:
bool vogelKollidiertMitBarriereAn (int posX, int posY) {

    if (posX > 150 && posX < 225) {
        if (vogel.getYCoord() < posY + 20 && vogel.getYCoord() > posY - 80) {
            /* Der Vogel kollidiert nicht: */
            return false;
        } else {
            return true;
        }
    }
    
    // Überprüft die obere und untere Schranke
    
    if (!alreadyCrashed && (vogel.getYCoord() + 20 > 400 || vogel.getYCoord() < 0)) {
        return true;
    }
}

In [ ]:
bool vogelKollidiert(Rect array1[], Rect array2[]) {
    
    int posX;
    int posY;
    
    // Wir müssen überprüfen, welche der Barrieren näher am Vogel liegt, die andere ist für die Kollisionsberechnung unerheblich:
    if (array1[rndm1].getX() < array2[rndm2].getX()) {
        
        // Die Koordinaten der ersten Barriere werden übergeben.
        
        posX = array1[rndm1].getX();
        posY = array1[rndm1].getY();
        
        return vogelKollidiertMitBarriereAn(posX, posY);
    } else {
        
        // Die Koordinaten der zweiten Barriere werden übergeben.
        
        posX = array2[rndm2].getX();
        posY = array2[rndm2].getY();
        
        return vogelKollidiertMitBarriereAn(posX, posY);
    }  
}

In [ ]:
// Wartet 3 Sekunden und fügt dann den Vogel ins Feld ein.
AlgoViz::sleep(3000);

layout.hideWelcome();
vogel.show();

In [ ]:
// Erstellt die erste Barriere

erstelleBarriere(barriere1);
rndm1 = rndm;

// Unsere Hauptschleife

while (key != "x" ) {
    /* Verarbeiten von Tastatureingaben; mit der ↑-Taste wird der Vogel nach oben beschleunigt,
       mit 'x' wird das Spiel beendet. */
    key = spielfeld.lastKey();
 
    if (key == "ArrowUp") {
        // Default Sprunghöhe ist -30 Pixel
        y += -30;
        ascend = true; 
    }

    /* Auf Kollision überprüfen */
    if (vogelKollidiert(barriere1, barriere2)) {
        vogelAbsturz();
    } else {
        score++;
    }
    
    /* Vertikale Geschwindigkeit verarbeiten */
    if (ascend) {
        vogel.move(y);
        ascension += -10;
        y += ascension;
        if (ascension <= -10) {
            ascend = false;
        }
    } else {
        // Erdanziehung: Die Beschleunigungsvariable wird um 3 erhöht, 9 ist der Schwellenwert
        if (ascension <= 9) {
            ascension += 3;
        }
        y += ascension;
        vogel.move(y);
    }
    
    // Rotiere den Vogel basierend auf seiner Steigung
    if (ascension < 0) { 
        vogel.rotate(-25);
    } else {  
        vogel.rotate(45);
    }
    
    // Wenn die erste Barrier den X-Wert 400 erreicht, wird die zweite Barriere erstellt (nur für den ersten Durchlauf relevant)
    if (!reachMid && barriere1[0].getX() == 400) {
        reachMid = true;
        erstelleBarriere(barriere2);
        rndm2 = rndm; 
    }
    
    // Wenn die Barrieren links am Bildschirm verschwinden, werden sie resettet
    if (barriere1[0].getX() == -100) {
        reseteBarriere(barriere1, rndm1, 1);
        for (int i = 0; i < 9; i++) {
           barriere1[i].moveTo(850, barriere1[i].getY()); 
        }
    } else if(barriere2[0].getX() == -100) {
        reseteBarriere(barriere2, rndm2, 2);
        for (int i = 0; i < 9; i++) { 
           barriere2[i].moveTo(850, barriere2[i].getY()); 
        }
    }
    
    // Bewegt die Barrieren
    for (int i = 0; i < 9; i++) {
       barriere1[i].moveTo((barriere1[i].getX() - 10), barriere1[i].getY()); 
       barriere2[i].moveTo((barriere2[i].getX() - 10), barriere2[i].getY());
    }
    
    screenScore = Text("Score: " + to_string(score / 7), -20, 15, &spielfeld);
    
    iteration++;
}